# _AutoAuto_ Self-driving Car

Let's use _Machine Learning_ (specifically, an _Artificial Neural Network_) to drive the _AutoAuto_ car around a track.

![](images/car.jpg)

### What is Machine Learning?

_Machine Learning_ (ML) is a subfield of _Artificial Intelligence_. ML algorithms use training data to learn patters. **We** write the learning algorithm, then we feed training data into the algorithm so that it can "learn".

ML is useful for:
1. Having our computers detect patters which we humans **can't** detect, or
2. Having our computers do tasks which are **boring** for us to do.

Read more on [Wikipedia](https://en.wikipedia.org/wiki/Machine_learning).

![](images/ml.jpg)

### Training Data

We collected over 8,000 images by manually driving the car around on the track. Every time an image was taken, the angle of the wheels was also recorded! That means we have over 8,000 image/angle pairs which were used to train the machine learning algorithm.

Here are a few examples from the training set. The number above each image denotes the angle of the wheels at the moment the image was taken (positive is _left_, negative is _right_).

![](images/samples.png)

### Preprocessing

Before using the training images, we _preprocess_ them to remove some of the noise which is irrelevent for driving.

See below how each of the examples images is preprocessed. Notice how we:
1. crop the image,
2. remove the color and,
3. only keep strong edges!

![](images/samples_preprocessed.png)

### What is an Artificial Neural Network?

An _Artificial Neural Network_ is one machine learning algorithm which usually does a good job of learning from image data like we have here!

Neural Networks contain many little neurons which are connected together into layers. Using an algorithm nicknamed "backprop", the neural network learns from the training data.

Read more on [Wikipedia](https://en.wikipedia.org/wiki/Machine_learning#Artificial_neural_networks).

![](images/ann_alt.png)

### Load the necessary Python libraries

The next cell loads the TensorFlow library. TensorFlow is a library which does very fast numeric operations. (Google primarily supports Tensorflow; thanks Google!)

In [1]:
from tensorflow import keras
from tensorflow.keras.models import load_model

/opt/berryconda3/bc3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The next cell loads the `OpenCV` library. OpenCV is used for doing image processing and computer vision. (Intel primarily supports OpenCV; thanks Intel!)

In [2]:
import numpy as np
import cv2

cv2.setNumThreads(0)   # <-- disable multithreading in opencv

The next cell loads the car library! (Thanks AutoAuto!)

In [3]:
import car
from car.motors import set_throttle, set_steering, CAR_THROTTLE_FORWARD_SAFE_SPEED

from auto.camera import CameraRGB

Plus we need a few other imports:

In [4]:
import time
from IPython.display import clear_output

### Load the Camera and the Neural Network!

The next cell creates a camera object and also loads the pre-trained Neural Network!

In [5]:
camera = CameraRGB()

model = load_model("model_01.hdf5")

### Create the Processing Functions

This next cell creates several functions to do the various steps of preprocessing. Read each function's docstring to learn what each function does.

In [6]:
from preprocessing import preprocess


def capture_preprocess_and_stream():
    """
    Captures, preprocesses, and streams an image from the camera. Returns the data that will be used
    by the Neural Network.
    """
    frame = camera.capture()
    frame_edge, frame_feats = preprocess(frame)
    car.stream(frame_edge, verbose=False)    # , to_labs=True
    return frame_feats

def make_prediction(frame_feats):
    """
    Use the Neural Network model to make a prediction for the angle of the wheels.
    """
    prediction = model.predict(frame_feats)[0][0]
    extremized_predition = prediction * 180.        # combat regression toward the mean
    return float(extremized_predition)              # convert from np.float to python's float

def loop_once():
    """
    Run one "loop" of the control algorithm, which does the following:
    1. Captures, preprocesses, and streams an image from the camera.
    2. Makes a prediction for the angle of the wheels, based on that image from the camera.
    3. Drives the car for a fraction of a second using that predicted angle.
    """
    frame_feats = capture_preprocess_and_stream()
    steering_angle = make_prediction(frame_feats)
    set_steering(steering_angle)
    clear_output(True)
    print("Angle:", steering_angle)
    set_throttle(CAR_THROTTLE_FORWARD_SAFE_SPEED)
    time.sleep(0.2)
    set_throttle(0)
    time.sleep(0.05)

### Finally, Drive the Car!

In [7]:
# We loop forever (via a while loop), and call `loop_once()` for each iteration.
try:
    while True:
        loop_once()

except KeyboardInterrupt:
    # This exception is raised and caught here when you press the STOP button.
    pass


# Since we're done with the loop above, ensure that the car stops by setting
# the throttle and steering below:
set_throttle(0)
set_steering(0)

Angle: -0.42884825728833675
